In [1]:
import torch
import numpy as np
import tiktoken

In [2]:
# input the any text to check the code execution
text = (
    "DFX : Low-latency FPGA Appliance for Accelerate Transformer based Text Generation"
)

# using the subword method to using tiktoken
enc = tiktoken.get_encoding("gpt2")
token_ids = enc.encode(text)
embedding_dim = 768  # model : GPT 2
np_embeddings = np.random.randn(len(token_ids), embedding_dim)

In [3]:
# embody to LN using NumPy
def numpy_layer_norm(x, eps=1e-5):
    mean = x.mean(axis=-1, keepdims=True)
    var = ((x - mean) ** 2).mean(axis=-1, keepdims=True)
    return (x - mean) / np.sqrt(var + eps)


normalized_np = numpy_layer_norm(np_embeddings)

In [4]:
# compare PyTorch(Checkint the work correctly the function, gamma and beta)
torch_input = torch.tensor(np_embeddings, dtype=torch.float32)
layer_norm = torch.nn.LayerNorm(embedding_dim)
normalized_torch = layer_norm(torch_input)

In [5]:
diff = np.abs(normalized_np - normalized_torch.detach().numpy()).mean()
print("compare diff:", diff)

compare diff: 3.530453860313632e-08
